In [1]:
import pandas as pd
import numpy as np

## 초등학교 위경도 변환

In [2]:
elementary=pd.read_csv('./data_busan/읍면동별_학교_부산.csv',encoding='cp949')
elementary.head(5)

,매장명,주소
0,안민초,부산 동래구 온천천로381번길 19
1,온천초,부산 동래구 충렬대로107번길 124
2,명륜초,부산 동래구 동래로 91
3,금강초,부산 동래구 사직북로28번길 125
4,안남초,부산 동래구 연안로 46


In [3]:
address=elementary['주소']
address.head(5)

0     부산 동래구 온천천로381번길 19
1    부산 동래구 충렬대로107번길 124
2           부산 동래구 동래로 91
3     부산 동래구 사직북로28번길 125
4           부산 동래구 연안로 46
Name: 주소, dtype: object

In [4]:
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

In [5]:
latitude = []
longitude =[]

for i in address:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])

In [6]:
address_ele=pd.DataFrame({'학교이름':elementary['매장명'],'주소':address,'위도':latitude,'경도':longitude})
address_ele.head(5)

,학교이름,주소,위도,경도
0,안민초,부산 동래구 온천천로381번길 19,0.000000,0.000000
1,온천초,부산 동래구 충렬대로107번길 124,35.210811,129.074106
2,명륜초,부산 동래구 동래로 91,35.211587,129.078863
3,금강초,부산 동래구 사직북로28번길 125,35.199954,129.062659
4,안남초,부산 동래구 연안로 46,35.193101,129.108051


### 초등학교 위경도->행정동 변환

In [7]:
import pandas as pd
import requests 
import sys
import json
import datetime 

In [8]:
def json_request(url='', encoding='utf-8', success=None, error=lambda e: print('%s : %s' % (e, datetime.now()), file=sys.stderr)):
    headers = {'Authorization': 'KakaoAK {}'.format(APP_KEY)}
    resp = requests.get(url, headers=headers)
    return resp.text

In [9]:
def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)
    # json request
    try:
        # print('try')
        json_req = json_request(url=url)
        json_data = json.loads(json_req)
        json_doc = json_data.get('documents')[1]
        json_name = json_doc.get('region_3depth_name')
        json_code = json_doc.get('code')
    except:
        # print('nan')
        json_name = 'NaN'
        json_code = 'NaN'
    return json_name, json_code

In [10]:
def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)
    # json request
    try:
        # print('try')
        json_req = json_request(url=url)
        json_data = json.loads(json_req)
        json_doc = json_data.get('documents')[1]
        json_name = json_doc.get('region_3depth_name')
        json_code = json_doc.get('code')
    except:
        # print('nan')
        json_name = 'NaN'
        json_code = 'NaN'
    return json_name, json_code
def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)
    # json request
    try:
        # print('try')
        json_req = json_request(url=url)
        json_data = json.loads(json_req)
        json_doc = json_data.get('documents')[1]
        json_name = json_doc.get('region_3depth_name')
        json_code = json_doc.get('code')
    except:
        # print('nan')
        json_name = 'NaN'
        json_code = 'NaN'
    return json_name, json_code
def get_address(x,y):
    address = []        
    json_name, json_code = reverse_geocode(x, y) 
    address.append(json_name)
    return address

In [11]:
def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)
    # json request
    try:
        # print('try')
        json_req = json_request(url=url)
        json_data = json.loads(json_req)
        json_doc = json_data.get('documents')[1]
        json_name = json_doc.get('region_3depth_name')
        json_code = json_doc.get('code')
    except:
        # print('nan')
        json_name = 'NaN'
        json_code = 'NaN'
    return json_name, json_code

In [12]:
def get_address(x,y):
    address = []        
    json_name, json_code = reverse_geocode(x, y) 
    address.append(json_name)
    return address

In [13]:
def get_code(x,y):
    code = []        
    json_name, json_code = reverse_geocode(x, y) 
    code.append(json_code)
    return code 

In [14]:
APP_KEY = 'dc125ad9de42a4ee620f0bdb2e623811' 
URL = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'

In [19]:
for i in range(len(address_ele)): 
    x_crd = float(address_ele.loc[i, '경도'])
    y_crd = float(address_ele.loc[i, '위도'])
    address = get_address(x_crd, y_crd)
    code = get_code(x_crd, y_crd)
    address_ele.loc[i, '행정동명'] = address
    address_ele.loc[i, '행정동_코드'] = code

    
    
    if i%100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

0 번째 경위도 좌표계에 대한 정보 불러오는 중...
100 번째 경위도 좌표계에 대한 정보 불러오는 중...
200 번째 경위도 좌표계에 대한 정보 불러오는 중...
300 번째 경위도 좌표계에 대한 정보 불러오는 중...
400 번째 경위도 좌표계에 대한 정보 불러오는 중...
500 번째 경위도 좌표계에 대한 정보 불러오는 중...


In [20]:
address_ele.head(5)

,학교이름,주소,위도,경도,행정동명,행정동_코드
0,안민초,부산 동래구 온천천로381번길 19,0.000000,0.000000,NaN,NaN
1,온천초,부산 동래구 충렬대로107번길 124,35.210811,129.074106,온천2동,2626056000
2,명륜초,부산 동래구 동래로 91,35.211587,129.078863,온천1동,2626055000
3,금강초,부산 동래구 사직북로28번길 125,35.199954,129.062659,사직1동,2626058000
4,안남초,부산 동래구 연안로 46,35.193101,129.108051,안락2동,2626075000


In [21]:
address_ele
for i in range(len(address_ele)):
    address_ele['행정동_코드'][i]=address_ele['행정동_코드'][i][0:8]
address_ele.head(5)

<ipython-input-21-e342b9d6db80>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address_ele['행정동_코드'][i]=address_ele['행정동_코드'][i][0:8]


,학교이름,주소,위도,경도,행정동명,행정동_코드
0,안민초,부산 동래구 온천천로381번길 19,0.000000,0.000000,NaN,NaN
1,온천초,부산 동래구 충렬대로107번길 124,35.210811,129.074106,온천2동,26260560
2,명륜초,부산 동래구 동래로 91,35.211587,129.078863,온천1동,26260550
3,금강초,부산 동래구 사직북로28번길 125,35.199954,129.062659,사직1동,26260580
4,안남초,부산 동래구 연안로 46,35.193101,129.108051,안락2동,26260750


In [22]:
address_ele.to_csv("./data/읍면동별_학교_부산.csv",encoding='cp949')

## 중학교 위경도 변환

In [22]:
middle=pd.read_csv('중학교.csv',encoding='cp949')
middle.head(5)

,매장명,주소
0,가락중학교,서울 송파구 송이로 45
1,가산중학교,서울 금천구 시흥대로115길 48
2,가원중학교,서울 송파구 중대로10길 40-18
3,가재울중학교,서울 서대문구 수색로 100-75
4,강남중학교,서울 동작구 대방동15길 29


In [23]:
address=middle['주소']
address.head(5)

0          서울 송파구 송이로 45
1     서울 금천구 시흥대로115길 48
2    서울 송파구 중대로10길 40-18
3     서울 서대문구 수색로 100-75
4       서울 동작구 대방동15길 29
Name: 주소, dtype: object

In [24]:
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

In [25]:
latitude = []
longitude =[]

for i in address:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])

In [26]:
address_middle=pd.DataFrame({'학교이름':middle['매장명'],'주소':address,'위도':latitude,'경도':longitude})
address_middle.head(5)

,학교이름,주소,위도,경도
0,가락중학교,서울 송파구 송이로 45,37.502576,127.116812
1,가산중학교,서울 금천구 시흥대로115길 48,37.468425,126.894431
2,가원중학교,서울 송파구 중대로10길 40-18,37.491318,127.124127
3,가재울중학교,서울 서대문구 수색로 100-75,37.577258,126.902785
4,강남중학교,서울 동작구 대방동15길 29,37.502543,126.922401


### 중학교 위경도->행정동 변환

In [27]:
for i in range(len(address_middle)): 
    x_crd = float(address_middle.loc[i, ['경도']])
    y_crd = float(address_middle.loc[i, ['위도']])
    address = get_address(x_crd, y_crd)
    code = get_code(x_crd, y_crd)
    address_middle.loc[i, ['행정동명']] = address
    address_middle.loc[i, ['행정동_코드']] = code
    
    if i%100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

0 번째 경위도 좌표계에 대한 정보 불러오는 중...
100 번째 경위도 좌표계에 대한 정보 불러오는 중...
200 번째 경위도 좌표계에 대한 정보 불러오는 중...
300 번째 경위도 좌표계에 대한 정보 불러오는 중...


In [28]:
address_middle
for i in range(len(address_middle)):
    address_middle['행정동_코드'][i]=address_middle['행정동_코드'][i][0:8]
address_middle.head(5)

C:\Users\82108\AppData\Local\Temp\ipykernel_14500\555862081.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address_middle['행정동_코드'][i]=address_middle['행정동_코드'][i][0:8]


,학교이름,주소,위도,경도,행정동명,행정동_코드
0,가락중학교,서울 송파구 송이로 45,37.502576,127.116812,송파2동,11710590
1,가산중학교,서울 금천구 시흥대로115길 48,37.468425,126.894431,독산1동,11545610
2,가원중학교,서울 송파구 중대로10길 40-18,37.491318,127.124127,가락본동,11710620
3,가재울중학교,서울 서대문구 수색로 100-75,37.577258,126.902785,북가좌1동,11410710
4,강남중학교,서울 동작구 대방동15길 29,37.502543,126.922401,대방동,11590660


In [29]:
address_middle.to_csv("읍면동별_중학교_서울.csv",encoding='cp949')

## 고등학교 위경도 변환

In [11]:
high=pd.read_csv('고등학교.csv',encoding='cp949')
high.head(5)

,매장명,주소
0,가락고등학교,서울 송파구 송이로 42
1,가재울고등학교,서울 서대문구 수색로 100-35
2,강동고등학교,서울 강동구 구천면로 572
3,강서고등학교,서울 양천구 목동중앙남로 27
4,강서공업고등학교,서울 강서구 방화대로47길 9


In [12]:
address=high['주소']
high.head(5)

,매장명,주소
0,가락고등학교,서울 송파구 송이로 42
1,가재울고등학교,서울 서대문구 수색로 100-35
2,강동고등학교,서울 강동구 구천면로 572
3,강서고등학교,서울 양천구 목동중앙남로 27
4,강서공업고등학교,서울 강서구 방화대로47길 9


In [13]:
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

In [14]:
latitude = []
longitude =[]

for i in address:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])

In [15]:
address_high=pd.DataFrame({'학교이름':high['매장명'],'주소':address,'위도':latitude,'경도':longitude})
address_high.head(5)

,학교이름,주소,위도,경도
0,가락고등학교,서울 송파구 송이로 42,37.502576,127.116812
1,가재울고등학교,서울 서대문구 수색로 100-35,37.577258,126.902785
2,강동고등학교,서울 강동구 구천면로 572,37.544691,127.113449
3,강서고등학교,서울 양천구 목동중앙남로 27,37.536826,126.866562
4,강서공업고등학교,서울 강서구 방화대로47길 9,37.576186,126.812705


### 고등학교 위경도->행정동 변환

In [14]:
for i in range(len(address_high)): 
    x_crd = float(address_high.loc[i, ['경도']])
    y_crd = float(address_high.loc[i, ['위도']])
    address = get_address(x_crd, y_crd)
    code = get_code(x_crd, y_crd)
    address_high.loc[i, ['행정동명']] = address
    address_high.loc[i, ['행정동_코드']] = code
    
    if i%100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

NameError: name 'address_high' is not defined

In [27]:
address_high
for i in range(len(address_high)):
    address_high['행정동_코드'][i]=address_high['행정동_코드'][i][0:8]
address_high.head(5)

C:\Users\82108\AppData\Local\Temp\ipykernel_22448\4031062845.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address_high['행정동_코드'][i]=address_high['행정동_코드'][i][0:8]


,학교이름,주소,위도,경도,행정동명,행정동_코드
0,가락고등학교,서울 송파구 송이로 42,37.502576,127.116812,송파2동,11710590
1,가재울고등학교,서울 서대문구 수색로 100-35,37.577258,126.902785,북가좌1동,11410710
2,강동고등학교,서울 강동구 구천면로 572,37.544691,127.113449,천호2동,11740610
3,강서고등학교,서울 양천구 목동중앙남로 27,37.536826,126.866562,목4동,11470540
4,강서공업고등학교,서울 강서구 방화대로47길 9,37.576186,126.812705,방화3동,11500641


In [28]:
address_high.to_csv("읍면동별_고등학교_서울.csv",encoding='cp949')

## 영화관 위경도 변환

In [43]:
theater=pd.read_csv('./data_busan/읍면동별_영화관_부산.csv',encoding='cp949')
theater.head(5)

,매장명,주소
0,CGV 부산명지,부산 강서구 명지국제8로10번길 38
1,롯데시네마 부산명지,부산 강서구 명지국제6로 107
2,노블랜드(롯데시네마 부산명지점,부산 강서구 명지국제6로 107
3,메가박스 부산대,부산 금정구 장전로12번길 55
4,롯데시네마 오투,부산 금정구 중앙대로 1607


In [44]:
address=theater['주소']
theater.head(5)

,매장명,주소
0,CGV 부산명지,부산 강서구 명지국제8로10번길 38
1,롯데시네마 부산명지,부산 강서구 명지국제6로 107
2,노블랜드(롯데시네마 부산명지점,부산 강서구 명지국제6로 107
3,메가박스 부산대,부산 금정구 장전로12번길 55
4,롯데시네마 오투,부산 금정구 중앙대로 1607


In [45]:
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

In [46]:
latitude = []
longitude =[]

for i in address:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])

In [47]:
address_theater=pd.DataFrame({'학교이름':theater['매장명'],'주소':address,'위도':latitude,'경도':longitude})
address_theater

,학교이름,주소,위도,경도
0,CGV 부산명지,부산 강서구 명지국제8로10번길 38,35.094233,128.902761
1,롯데시네마 부산명지,부산 강서구 명지국제6로 107,35.091679,128.903995
2,노블랜드(롯데시네마 부산명지점,부산 강서구 명지국제6로 107,35.091679,128.903995
3,메가박스 부산대,부산 금정구 장전로12번길 55,35.229477,129.088175
4,롯데시네마 오투,부산 금정구 중앙대로 1607,35.260760,129.091500
5,아이맥스DVD영화관 부산대점,부산 금정구 금정로60번길 24,35.229477,129.088175
6,제이제이디지털영화관,부산 금정구 장전로12번길 56,35.229477,129.088175
7,CGV 정관,부산 기장군 정관읍 정관중앙로 45,35.328015,129.182423
8,메가박스 정관,부산 기장군 정관읍 정관6로 39,35.319925,129.176121
9,롯데시네마 동부산아울렛,부산 기장군 기장읍 기장해안로 147,35.191800,129.212332


### 영화관 위경도->행정동 변환

In [50]:
for i in range(len(address_theater)): 
    x_crd = float(address_theater.loc[i, '경도'])  # 단일 라벨 사용
    y_crd = float(address_theater.loc[i, '위도'])  # 단일 라벨 사용
    address = get_address(x_crd, y_crd)
    code = get_code(x_crd, y_crd)
    address_theater.loc[i, '행정동명'] = address  # 단일 라벨 사용
    address_theater.loc[i, '행정동_코드'] = code  # 단일 라벨 사용
    
    if i % 100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

0 번째 경위도 좌표계에 대한 정보 불러오는 중...


In [52]:
address_theater
for i in range(len(address_theater)):
    address_theater['행정동_코드'][i]=address_theater['행정동_코드'][i][0:8]
address_theater.head(5)

<ipython-input-52-90f76d0c79bb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address_theater['행정동_코드'][i]=address_theater['행정동_코드'][i][0:8]


,학교이름,주소,위도,경도,행정동명,행정동_코드
0,CGV 부산명지,부산 강서구 명지국제8로10번길 38,35.094233,128.902761,명지1동,26440535
1,롯데시네마 부산명지,부산 강서구 명지국제6로 107,35.091679,128.903995,명지1동,26440535
2,노블랜드(롯데시네마 부산명지점,부산 강서구 명지국제6로 107,35.091679,128.903995,명지1동,26440535
3,메가박스 부산대,부산 금정구 장전로12번길 55,35.229477,129.088175,장전2동,26410610
4,롯데시네마 오투,부산 금정구 중앙대로 1607,35.260760,129.091500,남산동,26410670


In [53]:
address_theater.to_csv("./data/읍면동별_영화관_부산.csv",encoding='cp949')

## 다중이용시설 위경도 변환(위경도없는 자료)

In [48]:
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

In [49]:
uni=pd.read_csv('./data_busan/읍면동별_다중이용시설_위경도1_부산.csv',encoding='cp949')
uni.head(5)

,매장명,위도,경도
0,기술신용보증기금,35.146079,129.064165
1,(주)이마트 문현점,35.144303,129.064241
2,㈜ 이마트 문현점,35.144303,129.064241
3,IFC부산 오슬로파크,35.148285,129.065478
4,문현금융단지IFC부산 복합시설,35.148285,129.065478


In [35]:
address=uni['주소']
address.head(5)

0           부산 사하구 감천동 130
1    부산 기장군 정관읍 달산리 141-17
2       부산 기장군 철마면 고촌리 683
3     부산 기장군 장안읍 길천리 467-1
4        부산 기장군 철마면 웅천리 53
Name: 주소, dtype: object

In [36]:
latitude = []
longitude =[]

for i in address:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])

In [38]:
address_uni=pd.DataFrame({'학교이름':uni['매장명'],'주소':address,'위도':latitude,'경도':longitude})
address_uni.head(5)

,학교이름,주소,위도,경도
0,감천산림공원,부산 사하구 감천동 130,35.088586,129.004687
1,강변공원,부산 기장군 정관읍 달산리 141-17,0.000000,0.000000
2,거님길공원,부산 기장군 철마면 고촌리 683,0.000000,0.000000
3,고리동산,부산 기장군 장안읍 길천리 467-1,0.000000,0.000000
4,곰내재숲속쉼터,부산 기장군 철마면 웅천리 53,35.283465,129.150585


### 시설물수 위경도->행정동 변환(부산)

In [50]:
for i in range(len(uni)): 
    x_crd = float(uni.loc[i, '경도'])
    y_crd = float(uni.loc[i, '위도'])
    address = get_address(x_crd, y_crd)
    code = get_code(x_crd, y_crd)
    uni.loc[i, '행정동명'] = address
    uni.loc[i, '행정동_코드'] = code
    
    if i%100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

0 번째 경위도 좌표계에 대한 정보 불러오는 중...


In [51]:
uni
for i in range(len(uni)):
    uni['행정동_코드'][i]=uni['행정동_코드'][i][0:8]
uni.head(5)

<ipython-input-51-8bc5b284036b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni['행정동_코드'][i]=uni['행정동_코드'][i][0:8]


,매장명,위도,경도,행정동명,행정동_코드
0,기술신용보증기금,35.146079,129.064165,문현2동,26290690
1,(주)이마트 문현점,35.144303,129.064241,문현2동,26290690
2,㈜ 이마트 문현점,35.144303,129.064241,문현2동,26290690
3,IFC부산 오슬로파크,35.148285,129.065478,문현2동,26290690
4,문현금융단지IFC부산 복합시설,35.148285,129.065478,문현2동,26290690


In [52]:
uni.to_csv("읍면동별_시설물수_부산위경도1.csv",encoding='cp949')

## 대학교 위경도 변환

In [54]:
uni2=pd.read_csv('./data_busan/읍면동별_대학교_부산.csv',encoding='cp949')
uni2.head(5)

,매장명,주소
0,경남정보대학교 ICT관,부산 사상구 주례로 45
1,경남정보대학교 센텀캠퍼스,부산 해운대구 센텀중앙로 55
2,경상국립대학교 부산지구총동창회,부산 동래구 금강로 69
3,고신대학교 보건대학원,부산 서구 감천로 262
4,고신대학교 영도캠퍼스 벧엘1관,부산 영도구 와치로 185


In [55]:
address=uni2['주소']
address.head(5)

0       부산 사상구 주례로 45
1    부산 해운대구 센텀중앙로 55
2       부산 동래구 금강로 69
3       부산 서구 감천로 262
4      부산 영도구 와치로 185
Name: 주소, dtype: object

In [56]:
latitude = []
longitude =[]

for i in address:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])

In [58]:
address_uni2=pd.DataFrame({'학교이름':uni2['매장명'],'주소':address,'위도':latitude,'경도':longitude})
address_uni2.head(5)

,학교이름,주소,위도,경도
0,경남정보대학교 ICT관,부산 사상구 주례로 45,35.150292,129.011425
1,경남정보대학교 센텀캠퍼스,부산 해운대구 센텀중앙로 55,35.173078,129.129207
2,경상국립대학교 부산지구총동창회,부산 동래구 금강로 69,35.216106,129.078021
3,고신대학교 보건대학원,부산 서구 감천로 262,35.080911,129.014734
4,고신대학교 영도캠퍼스 벧엘1관,부산 영도구 와치로 185,35.088918,129.059352


### 대학교 위경도->행정동 변환(강동구)

In [59]:
for i in range(len(address_uni2)): 
    x_crd = float(address_uni2.loc[i, '경도'])
    y_crd = float(address_uni2.loc[i, '위도'])
    address = get_address(x_crd, y_crd)
    code = get_code(x_crd, y_crd)
    address_uni2.loc[i, '행정동명'] = address
    address_uni2.loc[i, '행정동_코드'] = code
    
    if i%100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))


0 번째 경위도 좌표계에 대한 정보 불러오는 중...


In [60]:
address_uni2
for i in range(len(address_uni2)):
    address_uni2['행정동_코드'][i]=address_uni2['행정동_코드'][i][0:8]
address_uni2.head(5)

<ipython-input-60-ebfa92e30e6c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address_uni2['행정동_코드'][i]=address_uni2['행정동_코드'][i][0:8]


,학교이름,주소,위도,경도,행정동명,행정동_코드
0,경남정보대학교 ICT관,부산 사상구 주례로 45,35.150292,129.011425,주례2동,26530660
1,경남정보대학교 센텀캠퍼스,부산 해운대구 센텀중앙로 55,35.173078,129.129207,우2동,26350520
2,경상국립대학교 부산지구총동창회,부산 동래구 금강로 69,35.216106,129.078021,온천1동,26260550
3,고신대학교 보건대학원,부산 서구 감천로 262,35.080911,129.014734,암남동,26140680
4,고신대학교 영도캠퍼스 벧엘1관,부산 영도구 와치로 185,35.088918,129.059352,청학2동,26200640


In [61]:
address_uni2.to_csv("읍면동별_대학교_부산.csv",encoding='cp949')